# Rapport : Dimensionnement d'un Buffer Réseau

<div align="center">

## Projet de Modélisation par Chaînes de Markov à Temps Discret

**Cours :** File d'attente, TSP FISA 2A  
**Auteurs :** Nassim EL HADDAD, Rebecca RINGUET  
**Date :** 2025

</div>

---

## Table des Matières

1. [Introduction et Contexte](#introduction)
2. [Modélisation du Système](#modelisation)
3. [Chaîne de Markov](#q1)
4. [Distribution Stationnaire](#q2)
5. [Métriques de Performance](#q3-q7)
6. [Analyse Multi-Capacités](#q8)
7. [Comparaison de Configurations](#q10)
8. [Conclusion](#conclusion)

---




In [ ]:
# Configuration et imports
import numpy as np
from scipy.linalg import solve
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import networkx as nx
import pandas as pd
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

# Configuration du style matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Couleurs personnalisées pour les graphiques
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'success': '#06A77D',
    'warning': '#F18F01',
    'danger': '#C73E1D',
    'info': '#6C5CE7'
}

print("Bibliothèques importées avec succès !")
print("Configuration des graphiques appliquée")


<a id="q10"></a>
## 9. Question 10 : Comparaison de Configurations

---

### Configuration Modifiée

Selon le sujet, nous modifions les probabilités d'arrivée : **p₀=0.5, p₁=0.2, p₂=0.3**

| Configuration | p₀ | p₁ | p₂ 
|---------------|----|----|----|----|
| **Originale** | 0.4 | 0.2 | 0.4 | 0.0 |
| **Modifiée** | 0.5 | 0.2 | 0.0 | 0.3 |

### Changements Clés

- **p₀ augmente** : Plus de slots sans arrivée (0.4 → 0.5)
- **p₂ diminue à 0** : Plus d'arrivées de 2 paquets (0.4 → 0.0)
- **p₂ réduit** : Nouvelles arrivées de 2 paquets (0.0 → 0.3)

### Impact Attendu

- **Débit d'arrivée théorique augmente** : 1.0 → 1.1 paquets/slot (0.5×0 + 0.2×1 + 0.0×2 + 0.3×3 = 1.1)
- **Plus de périodes creuses** : p₀ plus élevé permet au buffer de se vider plus souvent
- **Pics plus importants** : p₂ réduit des arrivées simultanées de 3 paquets, créant plus de congestion pour K petit


<a id="modelisation"></a>
## 2. Modélisation du Système

### Principe de la Chaîne de Markov

Une **chaîne de Markov à temps discret** est utilisée pour modéliser l'évolution du nombre de paquets dans le buffer au cours du temps.

#### États du Système
- **État i** : Nombre de paquets dans le buffer (i ∈ {0, 1, 2, ..., K})
- **K+1 états** possibles au total

#### Transitions d'État

Pour chaque état **i**, le système évolue selon :

1. **Service** : 1 paquet est servi (si i > 0)
   - État après service : $\max(0, i-1)$

2. **Arrivées** : 0, 1, 2 ou 3 paquets arrivent selon les probabilités
   - Nouvel état : $\min(K, \text{état\_après\_service} + \text{arrivées})$
   - Les paquets en excès sont **rejetés** si le buffer est plein

#### Matrice de Transition P

La matrice **P** de taille $(K+1) \times (K+1)$ représente les probabilités de transition :

$$P_{ij} = \mathbb{P}(X_{n+1} = j \mid X_n = i)$$

où $X_n$ est l'état du système au slot $n$.

### Métriques de Performance

Le projet calcule les métriques suivantes :

#### Distribution Stationnaire π

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue en résolvant :

$$\pi = \pi P \quad \text{avec} \quad \sum_{i=0}^{K} \pi_i = 1$$

#### Nombre Moyen de Clients L

$$L = \sum_{i=0}^{K} i \cdot \pi_i$$

#### Débit d'Entrée Xe

$$X_e = \sum_{i=0}^{K} \pi_i \cdot \mathbb{E}[\text{paquets acceptés} \mid \text{état } i]$$

#### Taux de Perte Loss

$$\text{Loss} = \sum_{i=0}^{K} \pi_i \cdot \mathbb{E}[\text{paquets perdus} \mid \text{état } i]$$

#### Temps de Réponse Moyen R (Formule de Little)

$$R = \frac{L}{X_e}$$

#### Taux d'Utilisation U

$$U = 1 - \pi_0$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien soit occupé.


In [ ]:
# Chargement des fonctions du projet
exec(open('buffer_dimensionnement.py').read())

print("Fonctions du projet chargées avec succès !")


<a id="q1"></a>
## 4. Question 1 : Chaîne de Markov

### Description de la Chaîne

Pour **K = 5**, la chaîne de Markov possède **6 états** (0 à 5) représentant le nombre de paquets dans le buffer.


In [ ]:
# Parametres pour la chaine de Markov
K_q1 = 5
p0, p1, p2 = 0.4, 0.2, 0.4

# Affichage direct de l'image de la chaine de Markov (centree)
display(HTML('<div style="text-align: center; margin: 20px 0;">'))
display(Image('graphes/chaineDeMarkov.jpeg', width=900))
display(HTML('</div>'))

print("\n**Commentaire :** Le graphe montre les 6 etats (0 a 5) et les transitions possibles.")
print("Les arcs sont etiquetes avec les probabilites de transition. On observe que :")
print("- L'etat 0 ne peut que recevoir des paquets (pas de service)")
print("- Les etats intermediaires peuvent recevoir ou perdre des paquets")
print("- L'etat 5 (buffer plein) peut perdre des paquets lors d'arrivees")


<a id="q2"></a>
## 5. Question 2 : Distribution de Probabilité Stationnaire π(K)

### Calcul de la Distribution Stationnaire

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue en résolvant le système d'équations :

$$\pi = \pi P$$

avec la contrainte de normalisation :

$$\sum_{i=0}^{K} \pi_i = 1$$

Ce système peut être réécrit sous la forme :

$$(P^T - I) \pi = 0$$

où $I$ est la matrice identité. En remplaçant la dernière équation par la contrainte de normalisation, on obtient un système linéaire résoluble.

Cette distribution représente les **probabilités d'équilibre** de chaque état du système à long terme, c'est-à-dire :

$$\pi_i = \lim_{n \to \infty} \mathbb{P}(X_n = i)$$
pport

In [ ]:
# Calcul des distributions stationnaires pour differentes valeurs de K
valeurs_K = [2, 5, 10, 15, 80]
resultats_original = []

for K in valeurs_K:
    pi = calculer_distribution_stationnaire(K, p0, p1, p2)
    resultats_original.append({
        'K': K,
        'pi': pi
    })

# Affichage direct du graphe de distribution stationnaire (centree)
display(HTML('<div style="text-align: center; margin: 30px 0;">'))
display(Image('graphes/Q2_distribution_stationnaire(Original).png', width=1000))
display(HTML('</div>'))

print("\n**Commentaire :** Les distributions stationnaires montrent la probabilite d'etre dans chaque etat en regime permanent.")
print("Observations :")
print("- Pour K petit (K=2, 5) : La probabilite est concentree sur les etats faibles")
print("- Pour K moyen (K=10, 15) : La distribution s'etale davantage")
print("- Pour K grand (K=80) : La distribution se stabilise avec une queue decroissante")
print("- L'etat 0 (buffer vide) a toujours une probabilite significative")


<a id="q3-q7"></a>
## 6. Questions 3-7 : Métriques de Performance

---

### Calcul des Métriques

Nous calculons maintenant toutes les métriques de performance pour chaque valeur de K :

#### Question 3 : Nombre Moyen de Clients L

$$L = \sum_{i=0}^{K} i \cdot \pi_i$$

Cette métrique représente le nombre moyen de paquets présents dans le système (buffer) en régime stationnaire.

#### Question 4 : Débit d'Entrée Xe

$$X_e = \sum_{i=0}^{K} \pi_i \cdot \sum_{a=0}^{A} p_a \cdot \min(a, K - \max(0, i-1))$$

où $A$ est le nombre maximum d'arrivées possibles (2 ou 3 selon la configuration) et $p_a$ est la probabilité d'avoir $a$ arrivées.

#### Question 5 : Nombre Moyen de Paquets Perdus Loss

$$\text{Loss} = \sum_{i=0}^{K} \pi_i \cdot \sum_{a=0}^{A} p_a \cdot \max(0, a - (K - \max(0, i-1)))$$

Les paquets sont perdus lorsque le buffer est plein et qu'il y a plus d'arrivées que d'espaces disponibles.

#### Question 6 : Temps de Réponse Moyen R (Formule de Little)

$$R = \frac{L}{X_e}$$

Cette formule fondamentale de la théorie des files d'attente relie le nombre moyen de clients dans le système, le débit d'entrée et le temps de réponse moyen.

#### Question 7 : Taux d'Utilisation du Lien U

$$U = 1 - \pi_0 = \sum_{i=1}^{K} \pi_i$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien de transmission soit occupé.


In [ ]:
# Calcul des metriques pour toutes les valeurs de K
resultats_complets = []

print("="*70)
print("CALCUL DES METRIQUES POUR CHAQUE VALEUR DE K")
print("="*70)

for K in valeurs_K:
    print(f"\n{'='*60}")
    print(f"ANALYSE POUR K = {K} (Questions Q2-Q7)")
    print(f"{'='*60}")
    resultat = analyser_buffer(K, p0, p1, p2, verbose=True)
    if resultat:
        resultats_complets.append(resultat)
    print()

# Extraction des valeurs pour utilisation dans d'autres cellules (variables globales)
# Ces variables sont accessibles dans toutes les cellules du notebook
K_values = [r['K'] for r in resultats_complets]
L_values = [r['L'] for r in resultats_complets]
Xe_values = [r['Xe'] for r in resultats_complets]
Loss_values = [r['Loss'] for r in resultats_complets]
R_values = [r['R'] for r in resultats_complets]
U_values = [r['U']*100 for r in resultats_complets]

# S'assurer que K_values est dans l'espace de noms global
globals()['K_values'] = K_values

# Tableau recapitulatif
df_resultats = pd.DataFrame({
    'K': [r['K'] for r in resultats_complets],
    'L (paquets)': [f"{r['L']:.6f}" for r in resultats_complets],
    'Xe (paquets/slot)': [f"{r['Xe']:.6f}" for r in resultats_complets],
    'Loss (paquets/slot)': [f"{r['Loss']:.6e}" for r in resultats_complets],
    'R (slots)': [f"{r['R']:.6f}" for r in resultats_complets],
    'U (%)': [f"{r['U']*100:.2f}" for r in resultats_complets]
})

# Affichage du tableau recapitulatif (centree et style ameliore)
display(HTML('''
<style>
.table-container {
    margin: 30px auto;
    max-width: 95%;
    overflow-x: auto;
}
table {
    margin: 0 auto;
    border-collapse: collapse;
    font-size: 14px;
}
table th {
    background-color: #2E86AB;
    color: white;
    padding: 12px;
    text-align: center;
    font-weight: bold;
}
table td {
    padding: 10px;
    text-align: center;
    border: 1px solid #ddd;
}
table tr:nth-child(even) {
    background-color: #f2f2f2;
}
table tr:hover {
    background-color: #e8f4f8;
}
</style>
<div class="table-container">
'''))
display(HTML(df_resultats.to_html(index=False, escape=False, 
                                   classes='table table-striped table-hover',
                                   table_id='resultats_table')))
display(HTML('</div>'))

# Affichage direct du graphe des metriques (centree)
display(HTML('<div style="text-align: center; margin: 30px 0;">'))
display(Image('graphes/Q3-Q7_metriques_K(Original).png', width=1000))
display(HTML('</div>'))

print("\n**Commentaire :** Les graphiques montrent l'evolution des metriques en fonction de K :")
print("- **L (nombre moyen de clients)** : Augmente avec K, plus de capacite = plus de paquets stockes")
print("- **Xe (debit d'entree)** : Augmente avec K, converge vers le debit theorique d'arrivee")
print("- **Loss (taux de perte)** : Diminue exponentiellement avec K (echelle logarithmique)")
print("- **R (temps de reponse)** : Augmente avec K, trade-off entre pertes et delai")
print("- **U (taux d'utilisation)** : Augmente avec K, reflete la probabilite que le buffer soit non vide")


<a id="q8"></a>
## 7. Question 8 : Analyse Multi-Capacités

---

### Analyse pour K ∈ {2, 5, 10, 15, 80}

Cette section présente une analyse comparative détaillée pour différentes capacités de buffer, incluant :
- L'état le plus probable pour chaque K
- Les probabilités que le buffer soit vide ou plein
- Une comparaison visuelle des métriques principales


### Tableau Récapitulatif des Métriques

---


In [ ]:
### Observations Principales

**Observations principales :**
- Pour K petit (K=2, 5) : Taux de perte eleve, debit d'entree limite
- Pour K moyen (K=10, 15) : Bon compromis entre pertes et delai
- Pour K grand (K=80) : Taux de perte negligeable, mais temps de reponse plus eleve

---


<a id="q9"></a>
## 8. Question 9 : Analyse et Commentaires sur les Résultats

---

### Observations Clés

#### Impact de la Capacité K sur les Performances

1. **Nombre moyen de clients (L)**
   - Augmente avec K : Plus de capacité = plus de paquets stockés en moyenne
   - Tendance à se stabiliser pour de grandes valeurs de K

2. **Débit d'entrée (Xe)**
   - Augmente avec K : Moins de rejets = plus de paquets acceptés
   - Converge vers le débit d'arrivée théorique pour K → ∞

3. **Taux de perte (Loss)**
   - Diminue exponentiellement avec K
   - Pour K petit : pertes importantes dues au buffer plein
   - Pour K grand : pertes négligeables

4. **Temps de réponse (R)**
   - Augmente avec K : Plus de paquets = temps d'attente plus long
   - Trade-off entre pertes et délai

5. **Taux d'utilisation (U)**
   - Augmente avec K : Le lien est utilisé plus souvent
   - Reflète la probabilité que le buffer soit non vide


<a id="q10"></a>
## 9. Question 10 : Comparaison de Configurations

---

### Configuration Modifiée

Selon le sujet, nous modifions les probabilités d'arrivée : **p₀=0.5, p₁=0.2, p₂=0.3**

| Configuration | p₀ | p₁ | p₂ |
|---------------|----|----|----|
| **Originale** | 0.4 | 0.2 | 0.4 |
| **Modifiée** | 0.5 | 0.2 | 0.3 |

### Changements Clés

- **p₀ augmente** : Plus de slots sans arrivée (0.4 → 0.5)
- **p₂ diminue** : Moins d'arrivées de 2 paquets (0.4 → 0.3)

### Impact Attendu

- **Débit d'arrivée théorique réduit** : 1.0 → 0.8 paquets/slot (0.5×0 + 0.2×1 + 0.3×2 = 0.8)
- **Plus de périodes creuses** : p₀ plus élevé permet au buffer de se vider plus souvent
- **Moins de pics** : p₂ réduit diminue les arrivées simultanées de 2 paquets


In [ ]:
# Calcul avec la configuration modifiee
p0_new, p1_new, p2_new = 0.5, 0.2, 0.3

print("="*70)
print("CALCUL DES METRIQUES POUR CHAQUE VALEUR DE K (CONFIGURATION MODIFIEE)")
print("="*70)

resultats_nouvelle = []
for K in valeurs_K:
    print(f"\n{'='*60}")
    print(f"ANALYSE POUR K = {K} (Questions Q2-Q7) - Configuration modifiee")
    print(f"{'='*60}")
    resultat = analyser_buffer(K, p0_new, p1_new, p2_new, verbose=True)
    if resultat:
        resultats_nouvelle.append(resultat)
    print()

# Extraction des valeurs pour visualisation (variables globales)
# S'assurer que K_values est défini globalement - utiliser resultats_complets si disponible
if 'resultats_complets' in globals() and len(resultats_complets) > 0:
    K_values = [r['K'] for r in resultats_complets]
else:
    # Fallback : utiliser valeurs_K si resultats_complets n'est pas disponible
    K_values = valeurs_K if 'valeurs_K' in globals() else [2, 5, 10, 15, 80]

# S'assurer que K_values est dans l'espace de noms global
globals()['K_values'] = K_values

L_orig = [r['L'] for r in resultats_complets]
Xe_orig = [r['Xe'] for r in resultats_complets]
Loss_orig = [r['Loss'] for r in resultats_complets]
R_orig = [r['R'] for r in resultats_complets]
U_orig = [r['U']*100 for r in resultats_complets]

L_new = [r['L'] for r in resultats_nouvelle]
Xe_new = [r['Xe'] for r in resultats_nouvelle]
Loss_new = [r['Loss'] for r in resultats_nouvelle]
R_new = [r['R'] for r in resultats_nouvelle]
U_new = [r['U']*100 for r in resultats_nouvelle]

# Affichage direct du graphe de comparaison (centree)
display(HTML('<div style="text-align: center; margin: 30px 0;">'))
display(Image('graphes/Q10_comparaison_configurations.png', width=1000))
display(HTML('</div>'))

print("\n**Commentaire :** La comparaison montre l'impact de la nouvelle distribution d'arrivees :")
print("- **Configuration modifiee** : Debit theorique reduit (0.8 vs 1.0 paquets/slot)")
print("- **Pour K petit** : Moins de congestion grace a moins d'arrivees simultanees")
print("- **Pour K grand** : Meilleure recuperation grace a p0=0.5 (plus de slots vides)")
print("- **Temps de reponse** : Generalement plus faible avec la configuration modifiee")


In [ ]:
# Le graphe de comparaison est affiche dans la cellule precedente


In [ ]:
# Analyse quantitative des differences
differences = []
for i in range(len(resultats_complets)):
    K = resultats_complets[i]['K']
    diff_L = ((L_new[i] - L_orig[i]) / L_orig[i] * 100) if L_orig[i] > 0 else 0
    diff_Xe = ((Xe_new[i] - Xe_orig[i]) / Xe_orig[i] * 100) if Xe_orig[i] > 0 else 0
    diff_Loss = ((Loss_new[i] - Loss_orig[i]) / Loss_orig[i] * 100) if Loss_orig[i] > 0 else 0
    diff_R = ((R_new[i] - R_orig[i]) / R_orig[i] * 100) if R_orig[i] > 0 else 0
    diff_U = ((U_new[i] - U_orig[i]) / U_orig[i] * 100) if U_orig[i] > 0 else 0
    
    differences.append({
        'K': K,
        'Delta L (%)': f"{diff_L:+.2f}",
        'Delta Xe (%)': f"{diff_Xe:+.2f}",
        'Delta Loss (%)': f"{diff_Loss:+.2f}",
        'Delta R (%)': f"{diff_R:+.2f}",
        'Delta U (%)': f"{diff_U:+.2f}"
    })

df_differences = pd.DataFrame(differences)
display(HTML(df_differences.to_html(index=False, escape=False,
                                   classes='table table-striped table-hover',
                                   table_id='differences_table')))

# Calcul du debit theorique pour les deux configurations
debit_theorique_orig = p0*0 + p1*1 + p2*2
debit_theorique_new = p0_new*0 + p1_new*1 + p2_new*2

print(f"\nDebits theoriques d'arrivee :")
print(f"   - Configuration originale : {debit_theorique_orig:.4f} paquets/slot")
print(f"   - Configuration nouvelle  : {debit_theorique_new:.4f} paquets/slot")
print(f"   - Difference              : {debit_theorique_new - debit_theorique_orig:+.4f} paquets/slot")

print(f"\nObservations principales :")
print(f"   - La nouvelle configuration a un debit d'arrivee theorique plus eleve")
print(f"   - Les pics de 3 paquets creent plus de congestion pour K petit")
print(f"   - Les periodes creuses (p0=0.5) permettent une meilleure recuperation")
print(f"   - Pour K grand, l'impact des pics est attenue")


<a id="conclusion"></a>
## 10. Conclusion et Recommandations

### Synthèse des Résultats

#### Impact de la Capacité K

1. **Petites capacités (K = 2, 5)**
   - Taux de perte élevé
   - Débit d'entrée limité
   - Temps de réponse faible (peu de paquets en attente)
   - **Recommandation** : Inadapté pour des applications sensibles aux pertes

2. **Capacités moyennes (K = 10, 15)**
   - Bon compromis entre pertes et délai
   - Débit d'entrée proche du théorique
   - Temps de réponse acceptable
   - **Recommandation** : Optimal pour la plupart des applications

3. **Grandes capacités (K = 80)**
   - Taux de perte négligeable
   - Débit d'entrée maximal
   - Temps de réponse plus élevé
   - **Recommandation** : Adapté pour applications tolérantes au délai mais sensibles aux pertes

#### Comparaison des Configurations

- **Configuration originale** : Plus stable, moins de pics
- **Configuration modifiée** : Pics plus intenses mais périodes de récupération plus longues
- **Recommandation** : Choisir selon les contraintes de l'application

### Recommandations Finales

1. **Pour minimiser les pertes** : Utiliser K ≥ 15
2. **Pour minimiser le délai** : Utiliser K ≤ 10
3. **Pour un compromis optimal** : K = 10-15
4. **Pour la configuration** : Préférer l'originale pour plus de stabilité

---

<div align="center">

### Projet réalisé dans le cadre du cours de File d'attente, TSP FISA 2A

**Auteurs :**  
Nassim EL HADDAD  
Rebecca RINGUET

**Merci de votre attention !**

</div>
